In [4]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF

import json
import numpy as np
import pandas as pd

In [27]:
mapbox_access_token = 'pk.eyJ1IjoiZGxyb21hbm9mZiIsImEiOiJjanBteXZycnQwNGFiNDRwYnVqMTMxOWxnIn0.NDZecgSdD6iH8CK0XXdx0g'

In [30]:
df = pd.read_csv('https://raw.githubusercontent.com/DanielleRomanoff/Organize_Data_Blog/master/world_earthquakes_last_30.csv')
df = df.drop(['nst', 'gap', 'dmin', 'horizontalError', 'magError', 'magNst'], axis=1)

In [33]:
quake = df.copy()
quake = quake.drop(['magType', 'rms', 'net', 'id', 'updated', 'type', 'depthError', 'status',
                 'locationSource', 'magSource', 'place'], axis=1)
quake.head()

,time,latitude,longitude,depth,mag
0,2019-05-28T16:18:18.190Z,19.455999,-155.59816,-1.32,2.71
1,2019-05-28T15:35:39.538Z,48.719200,153.37620,122.19,4.80
2,2019-05-28T15:31:30.080Z,18.206800,-67.72480,10.00,2.77
3,2019-05-28T15:00:27.157Z,-8.623900,108.64260,98.83,4.70
4,2019-05-28T14:41:45.039Z,-8.908500,-74.53690,148.39,4.50


In [34]:
quake['time'] = quake['time'].str.extract('(\d\d\d\d-\d\d-\d\d)', expand=True)
quake.head()

,time,latitude,longitude,depth,mag
0,2019-05-28,19.455999,-155.59816,-1.32,2.71
1,2019-05-28,48.719200,153.37620,122.19,4.80
2,2019-05-28,18.206800,-67.72480,10.00,2.77
3,2019-05-28,-8.623900,108.64260,98.83,4.70
4,2019-05-28,-8.908500,-74.53690,148.39,4.50


In [40]:
site_lat = quake.latitude
site_lon = quake.longitude
locations_name = df.place

data = [
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=(quake.mag.values*5),
            color='rgb(255, 0, 0)',
            opacity=0.7
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=(quake.mag.values*2),
            color='rgb(242, 177, 172)',
            opacity=0.7
        ),
        hoverinfo='none'
    )]
        
layout = go.Layout(
    title='Earthquakes in the last 30 Days',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Earthquakes in the last 30 Days')